# NEURON modeling exercise #3

## NEURON (with Python)
* NEURON documentation: https://www.neuron.yale.edu/neuron/static/py_doc/index.html
* NEURON + Python tutorial: https://neuron.yale.edu/neuron/static/docs/neuronpython/index.html

## More on Sections

In case that you want to build an artificial neuron model, instead of using the experimentally measured morphology, you can build a multi-compartmental model by constructing Sections and connecting them.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h, gui

# we build a three-compartment model
soma = h.Section(name='soma')
dends = [h.Section(name=f'dend{i+1}') for i in range(2)]

In [ ]:
print(dends)
dends[0].psection()

Let's adjust their size first.

In [ ]:
soma.L = 4
soma.diam = 8

for d in dends:
    d.L = 600
    d.diam = 5
    d.nseg = 1 # isopotential sections

Then, we connect the soma and dend(rite)s electrically.

In [ ]:
for d in dends:
    d.connect(soma(1), 0)

In [ ]:
h.topology()

Let's include a passive mechanism to the soma and dendrites.

In [ ]:
for s in h.allsec():
    s.insert('pas')
    s.g_pas = 0.5e-4

h.v_init = soma.e_pas # we set the initial voltage to the reversal potential of the passive membrane


Let's put a current clamp on one of the dendrites.

In [ ]:
ic = h.IClamp(dends[0](0.5))
ic.amp = 0.2 # 200 pA
ic.delay = 10
ic.dur = 150

In [ ]:
def record_and_run(tstop=250, dt=0.1):
    vrec = h.Vector()
    trec = h.Vector()
    
    vrec.record(soma(0.5)._ref_v, dt)
    trec.record(h._ref_t, dt)
    
    h.tstop = tstop
    h.init()
    h.run()

    return vrec.c(), trec.c()
    
def run_and_plot(tstop=250, ax=None):
    v, t = record_and_run(tstop=tstop)
    
    if ax is None:
        _, ax = plt.subplots()
    ax.plot(t, v)

    

In [ ]:
run_and_plot()

Let's change the coupling of the dendrite and soma. We first measure what is the resistance between the soma and the first dendrite.

In [ ]:
# https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/topology/geometry.html?highlight=ri#ri
# Return the resistance (in megohms) between the center of the segment section(x) and its parent segment.

dends[0](0.5).ri()

We can change the result by changing the axial resistance of the dendrite.

In [ ]:
_, ax = plt.subplots()

v, t = record_and_run()
ax.plot(t, v, label=f'{dends[0].Ra}')

dends[0].Ra = dends[1].Ra*10
print(f'Now the resistance is {dends[0](0.5).ri():.4f} MΩ.')

v, t = record_and_run()
ax.plot(t, v, label=f'{dends[0].Ra}')
_ = plt.legend()

## Active mechanisms and NMODL

* NEURON Extension to NMODL: https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/nmodl2.html
* NMODL: https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/nmodl.html#nmodl

### How to compile mechanisms:
1. `nrnivmodl mod.files` in Terminal (or `cd mod.files; nrnivmodl`), OR
2. Use `mknrndll.app` (or equivalent apps in Windows or Linux).

In [ ]:
%%bash
nrnivmodl mod.files

### How to load mechanisms:
1. Place `x86_64` directory in the same directory as the simulation, OR
2. Use "File" -> "load dll" in GUI, OR
3. Use `neuron.load_mechanisms('directory name')`.


Now, we reuse the three-compartment cell but move the current clamp to the soma. Then, we insert the "hh2" mechanism (in `mod.files/HH2.mod`) and check the cells

In [ ]:
ic.loc(soma(0.5))

soma.insert("hh2")

# We set the temperature to 35°C so that hh2 can work properly
h.celsius = 35

h.psection()

In [ ]:
ic.dur = 300
ic.amp = 0.2 # 200 pA

In [ ]:
def record_and_run(tstop=250, dt=0.1):
    vrec1 = h.Vector()
    vrec2 = h.Vector()
    vrec3 = h.Vector()

    trec = h.Vector()
    
    vrec1.record(soma(0.5)._ref_v, dt)
    vrec2.record(dends[0](0.5)._ref_v, dt)
    vrec3.record(dends[1](0.5)._ref_v, dt)

    trec.record(h._ref_t, dt)
    
    h.tstop = tstop
    h.init()
    h.run()

    return vrec1.c(), vrec2.c(), vrec3.c(), trec.c()
    
def run_and_plot(tstop=250, ax=None):
    v1, v2, v3, t = record_and_run(tstop=tstop)
    
    if ax is None:
        _, ax = plt.subplots()
    ax.plot(t, v1, label='soma')
    ax.plot(t, v2, label='dend1')
    ax.plot(t, v3, label='dend2')
    
    _ = plt.legend()

In [ ]:
# soma.gnabar_hh2 = 0.003*100
# soma.gkbar_hh2 = 0.005*150

run_and_plot()

Here, we add a `NetCon` object that will monitor the membrane potential of the cell and detect events with `v` crossing -20 mV. Since this `NetCon` does not need to deliver events to anything else, we connect it to `None`. Then, we can record the event times to a vector, which lets us to record spike times, detected by a voltage threshold.

In [ ]:
nc = h.NetCon(soma(0.5)._ref_v, None, -20, 0, 1)
tspike = h.Vector()
nc.record(tspike)

In [ ]:
run_and_plot()

Here we print the spike times. Note that we used `.as_numpy` to transform a NEURON `Vector` to a numpy array

In [ ]:
print(tspike.as_numpy())

## Pyramidal cell example

Now let's simulate the fully active pyramidal cell model. **Please make sure that you restart the kernel at this point.**

In [ ]:
import neuron
from neuron import h, gui
import libcell

import numpy as np
import matplotlib.pyplot as plt

First we defined a passive cell,

In [ ]:
cell = libcell.L23()

In [ ]:
neuron.load_mechanisms("mod.files")

And then, we call a function in `libcell.py` to embed active mechanisms everywhere,`

In [ ]:
libcell.init_active(cell, axon=True, soma=True, dend=True, dendNa=True, dendCa=True)
h.psection()

To test the excitability of the cell, we add a current clamp electrode at soma,

In [ ]:
ic = h.IClamp(cell.soma(0.5))

Let's define a function to do a virtual current clamp experiment,

In [ ]:
def do_current_clamp(current_injected):
    
    # should not forget setting the temperature...
    h.celsius = 35
    
    # Simulation length = 300 ms
    h.tstop = 300

    # Current injection for 200 ms
    ic.delay = 50
    ic.dur = 200
    ic.amp = current_injected


    dt_rec = 0.1
    t = h.Vector()
    v = h.Vector()
    v.record(cell.soma(1)._ref_v, dt_rec)
    t.record(h._ref_t, dt_rec)

    # We set the initial voltage to -75 mV and run the simulation
    h.v_init = -75
    h.init()
    h.run()
    
    # Here we plot the result
    _, ax = plt.subplots()

    ax.plot(t, v)
    ax.set(xlabel="time (ms)", ylabel = "v (mv)")

In [ ]:
do_current_clamp(0.1) # 100 pA injection; the cell is still in a subthreshold regime

In [ ]:
do_current_clamp(0.35) # 350 pA injection will give you spikes